In [119]:
import numpy as np
import cv2
import yaml
from math import *
import matplotlib.pyplot as plt
import os
import glob
chessboardsize = (10,6)

In [120]:
# termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

chessboardsize = (10,6)
frame_size = (720, 1600)

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((chessboardsize[0] * chessboardsize[1],3), np.float32)
objp[:,:2] = np.mgrid[0:chessboardsize[0],0:chessboardsize[1]].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.

In [121]:
os.chdir(r'C:\Users\dell\camcal\customChessboardCalibration\chessboardImages')
images = glob.glob('*.jpg')
for image in images:
    img = cv2.imread(image)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    ret, corners = cv2.findChessboardCorners(gray, chessboardsize, None)

    if ret == True:
        objpoints.append(objp)
        corners2 = cv2.cornerSubPix(gray, corners, (11,11), (-1,-1), criteria)
        imgpoints.append(corners2)

        
        #Draw chess board corners
        cv2.drawChessboardCorners(img, chessboardsize, corners2, ret)
 

In [122]:
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, 
                                                   gray.shape[::-1], None, None)

In [123]:
len(imgpoints)

10

In [124]:
mtx, dist

(array([[1.16053179e+03, 0.00000000e+00, 3.47154264e+02],
        [0.00000000e+00, 1.15044497e+03, 8.06203814e+02],
        [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]]),
 array([[ 1.22323383e-01, -4.96427131e-01, -7.72714868e-04,
         -4.26708240e-04,  6.85348945e-01]]))

## k1=0.12232, k2=-0.49642, p1=-0.00077, p2=-0.00042, k3=0.68534 

In [125]:
new_mtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, imageSize = tuple(frame_size), 
                                             alpha = 1, newImgSize=tuple(frame_size))

In [126]:
new_mtx, roi

(array([[1.16945731e+03, 0.00000000e+00, 3.47122540e+02],
        [0.00000000e+00, 1.17178333e+03, 8.04480353e+02],
        [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]]),
 (8, 10, 702, 1580))

## Removing distortions from images and saving them

In [113]:
for i, image in enumerate(images):
    
    os.chdir(r'C:\Users\dell\camcal\customChessboardCalibration\chessboardImages')
    img = cv2.imread(image)
    
    # undistort
    dst = cv2.undistort(img, mtx, dist, None, new_mtx)
    # crop the image
    x, y, w, h = roi
    dst = dst[y:y+h, x:x+w]
    os.chdir(r'C:\Users\dell\camcal\customChessboardCalibration\chessboardImages\UndistortedImages')
    cv2.imwrite(f'undistorted_{i}.jpg', dst)
    

In [144]:
h, w = dst.shape[:2]
h, w, roi, x, y

(1580, 702, (8, 10, 702, 1580), 8, 10)

In [149]:
# Realigning centre wrt roi

cx = new_mtx[0][2].astype(np.int32)
cy = new_mtx[1][2].astype(np.int32)
fx = new_mtx[0][0].astype(np.int32)
fy = new_mtx[1][1].astype(np.int32)
print(cx,cy)
cx = cx - x
cy = cy - y
cx, cy

347 804


(339, 794)

## Distorting Images

In [211]:
dist_assumed = np.array([[ 1.22323383e-01, -4.96427131e-01, -7.72714868e-04,
         -4.26708240e-04,  6.85348945e-01]])
dist_assumed = dist_assumed * 3
dist_assumed

array([[ 3.66970149e-01, -1.48928139e+00, -2.31814460e-03,
        -1.28012472e-03,  2.05604684e+00]])

In [212]:
def dist_point(x, y):
    k1 = dist_assumed[0][0]
    k2 = dist_assumed[0][1]
    p1 = dist_assumed[0][2]
    p2 = dist_assumed[0][3]
    k3 = dist_assumed[0][4]
    
    
    x = (x - cx)/fx
    y = (y - cy)/fy
    r = (x**2 + y**2) ** 0.5
    
    xd = x * (1 + k1*(r**2)+ k2*(r**4) + k3*(r**6))
    yd = y * (1 + k1*(r**2)+ k2*(r**4) + k3*(r**6))
    xd = xd+(2*p1*xd*yd + p2*(r**2 + 2*(xd**2)))
    yd = yd+(p1*(r**2 + 2*(yd**2))+ 2*p2*xd*yd)
    
    x = x*fx + cx
    y = y*fy + cy
    xd = xd*fx + cx
    yd = yd*fy +cy
    x, y, xd, yd = int(x), int(y), int(xd), int(yd)
    return xd, yd

def distort_img(img):
    new_img = np.zeros(shape = img.shape, dtype = np.int32)
    h = new_img.shape[0]
    w = new_img.shape[1]
    for i in range(h):
        for j in range(w):
             
            dist_j, dist_i = dist_point(j, i)
            if (dist_j<0 or dist_i<0) or (dist_j>w-1 or dist_i>h-1):
                #Cannot be mapped
               
                pass
            else:
                
                new_img[dist_i][dist_j] = img[i][j]
    return new_img

In [213]:
os.chdir(r'C:\Users\dell\camcal\customChessboardCalibration\chessboardImages\UndistortedImages')
images = glob.glob('*.jpg')

for i, image in enumerate(images):
    os.chdir(r'C:\Users\dell\camcal\customChessboardCalibration\chessboardImages\UndistortedImages')
    img = cv2.imread(image)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    imgDist = distort_img(img)
    
    os.chdir(r'C:\Users\dell\camcal\DistortedImages2')
    cv2.imwrite(f'Distorted_Cropped_image_{i}.jpg', imgDist)

## Using distorted images & calibrating them

In [214]:
os.chdir(r'C:\Users\dell\camcal\DistortedImages2')
dist_images = glob.glob('*.jpg')

# termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

chessboardsize = (10,6)
frame_size = w, h

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((chessboardsize[0] * chessboardsize[1],3), np.float32)
objp[:,:2] = np.mgrid[0:chessboardsize[0],0:chessboardsize[1]].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.
for image in dist_images:
    img = cv2.imread(image)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    ret, corners = cv2.findChessboardCorners(gray, chessboardsize, None)

    if ret == True:
        objpoints.append(objp)
        corners2 = cv2.cornerSubPix(gray, corners, (11,11), (-1,-1), criteria)
        imgpoints.append(corners2)

        
        #Draw chess board corners
        cv2.drawChessboardCorners(img, chessboardsize, corners2, ret)
        
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, 
                                                   gray.shape[::-1], None, None)

In [215]:
len(imgpoints)

9

In [216]:
dist_assumed

array([[ 3.66970149e-01, -1.48928139e+00, -2.31814460e-03,
        -1.28012472e-03,  2.05604684e+00]])

In [217]:
dist

array([[ 0.30546097, -0.86156249, -0.0063288 , -0.0077803 ,  1.02091076]])

In [31]:
[k1, k2, p1, p2, k3]

[0.3, -1.28, -0.001, 0.0002, 1.78]

In [28]:
dist

array([[ 0.40209624, -0.93054953, -0.0163314 , -0.09411783,  1.16097643]])